<a href="https://colab.research.google.com/github/dsrushton/VGG_Object_Detector/blob/main/Copy_of_GemmAA_2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r "/content/drive/MyDrive/gemma-2-2b-it" /content/

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the tokenizer and model from your local path
model_path = "/content/drive/MyDrive/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2SdpaAttention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), 

In [ ]:
import os

data_path = "/content/drive/Othercomputers/My Computer (1)/Google Drive Sync/Shared Docs/Cleaned"

# Load and concatenate all text files in the directory
texts = []
for filename in os.listdir(data_path):
    if filename.endswith(".txt"):
        with open(os.path.join(data_path, filename), "r", encoding="utf-8") as file:
            texts.append(file.read())

# Combine all texts into one large string
combined_text = " ".join(texts)

# Tokenize the text
inputs = tokenizer(combined_text, return_tensors="pt", max_length=1024, truncation=True)

//Reg Mem\\

In [ ]:
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset

class TextDataset(Dataset):
    def __init__(self, inputs):
        self.inputs = inputs.input_ids

    def __len__(self):
        return self.inputs.shape[0]

    def __getitem__(self, idx):
        return self.inputs[idx]

# Prepare the dataset and dataloader
dataset = TextDataset(inputs)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)  # Adjust batch size as needed

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Enable mixed precision training
scaler = torch.cuda.amp.GradScaler()

# Fine-tuning loop
model.train()
for epoch in range(3):  # Adjust number of epochs
    for batch in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast('cuda'):
            outputs = model(input_ids=batch, labels=batch)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        print(f"Loss: {loss.item()}")

model.save_pretrained("/content/drive/MyDrive/MTLiens")
tokenizer.save_pretrained("/content/drive/MyDrive/MTLiens")



<ipython-input-4-69c8c3731d06>:22: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
<ipython-input-4-69c8c3731d06>:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


OutOfMemoryError: CUDA out of memory. Tried to allocate 82.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 52.81 MiB is free. Process 26375 has 39.50 GiB memory in use. Of the allocated memory 38.47 GiB is allocated by PyTorch, and 544.64 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

//Low Mem\\

In [ ]:
import os
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

class TextDataset(Dataset):
    def __init__(self, inputs):
        self.inputs = inputs.input_ids

    def __len__(self):
        return self.inputs.shape[0]

    def __getitem__(self, idx):
        return self.inputs[idx]

# Prepare the dataset and dataloader
dataset = TextDataset(inputs)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)  # Adjust batch size as needed

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Enable mixed precision training
scaler = torch.cuda.amp.GradScaler()

# Accumulate gradients over 4 steps
gradient_accumulation_steps = 4

# Fine-tuning loop with gradient accumulation
model.train()
for epoch in range(3):  # Adjust number of epochs
    for i, batch in enumerate(dataloader):
        batch = batch.to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(input_ids=batch, labels=batch)
            loss = outputs.loss

        # Scale the loss and accumulate gradients
        loss = loss / gradient_accumulation_steps
        scaler.scale(loss).backward()

        # Update weights every gradient_accumulation_steps
        if (i + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        print(f"Loss: {loss.item()}")

model.save_pretrained("/content/drive/MyDrive/MTLiens")
tokenizer.save_pretrained("/content/drive/MyDrive/MTLiens")
torch.save(model.state_dict(), "/content/drive/MyDrive/MTLiens/pytorch_model.bin")

<ipython-input-17-1924855c1756>:25: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
<ipython-input-17-1924855c1756>:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Loss: 0.8589140772819519
Loss: 0.8589140772819519
Loss: 0.8589140772819519


('/content/drive/MyDrive/MTLiens/tokenizer_config.json',
 '/content/drive/MyDrive/MTLiens/special_tokens_map.json',
 '/content/drive/MyDrive/MTLiens/tokenizer.model',
 '/content/drive/MyDrive/MTLiens/added_tokens.json',
 '/content/drive/MyDrive/MTLiens/tokenizer.json')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the original model
original_model_path = "/content/drive/MyDrive/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(original_model_path)
model = AutoModelForCausalLM.from_pretrained(original_model_path)

# Load the fine-tuned model's weights
fine_tuned_model_path = "/content/drive/MyDrive/MTLiens" # Update with your fine-tuned model path
state_dict = torch.load(f"{fine_tuned_model_path}/pytorch_model.bin")
model.load_state_dict(state_dict)

# Save the model
model.save_pretrained("/content/drive/MyDrive/MTLiens/Refined") # Update with desired save path
tokenizer.save_pretrained("/content/drive/MyDrive/MTLiens/Refined") # Update with desired save path

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-2-4a95b06148a7>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f"{fine_tuned_model_path}/pytorch_model.bin")


('/content/drive/MyDrive/MTLiens/Refined/tokenizer_config.json',
 '/content/drive/MyDrive/MTLiens/Refined/special_tokens_map.json',
 '/content/drive/MyDrive/MTLiens/Refined/tokenizer.model',
 '/content/drive/MyDrive/MTLiens/Refined/added_tokens.json',
 '/content/drive/MyDrive/MTLiens/Refined/tokenizer.json')

In [ ]:
# Define paths
model_path = "/content/drive/MyDrive/MTLiens/Refined"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2SdpaAttention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), 

In [ ]:
# Example input text (prompt)
input_text = "What is the cure for alcoholism?"

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt").to(device)

In [ ]:
# Generate text
outputs = model.generate(
    inputs['input_ids'],
    max_length=500,  # Adjust max_length based on your needs
    num_return_sequences=1,
    temperature=0.7,  # Adjust for more creative (higher) or deterministic (lower) output
    top_k=50,
    top_p=0.95,
    do_sample=True
)

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


What is the cure for alcoholism?

There is no one-size-fits-all answer to this question.  Alcoholism is a complex medical condition that requires professional help and individualized treatment. However, here's a general overview of the path to recovery:

**1. Recognizing the Problem:**

* **Acknowledge the problem:** The first step is to admit that there's a problem with alcohol use. This can be difficult, but it's essential for starting the recovery process.
* **Identify triggers:**  Understanding what situations or emotions lead to excessive drinking is crucial for managing triggers and avoiding relapse.
* **Seek support:**  Talk to friends, family, or a therapist about your struggles. Having a support system can make a significant difference.

**2. Seeking Professional Help:**

* **Consult a doctor:** A medical professional can assess the severity of alcohol dependence and recommend appropriate treatment options.
* **Consider rehab:**  Residential treatment programs offer a structur

In [ ]:
# Example input text (prompt)
input_text = "What is the 7th step?"

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate text
outputs = model.generate(
    inputs['input_ids'],
    max_length=500,  # Adjust max_length based on your needs
    num_return_sequences=1,
    temperature=0.7,  # Adjust for more creative (higher) or deterministic (lower) output
    top_k=50,
    top_p=0.95,
    do_sample=True
)

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.20 GiB. GPU 0 has a total capacity of 39.56 GiB of which 22.81 MiB is free. Process 106027 has 39.53 GiB memory in use. Of the allocated memory 37.82 GiB is allocated by PyTorch, and 1.21 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

//Claude Iteration\\

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from torch.nn.utils.rnn import pad_sequence
import os
from torch.optim import AdamW

# Load the tokenizer and model from your local path
model_path = "/content/drive/MyDrive/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, attn_implementation='eager')

# Move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

data_path = "/content/drive/Othercomputers/My Computer (1)/Google Drive Sync/Shared Docs/Cleaned"
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/gemma-2-2b-it")

# Load and concatenate all text files in the directory
combined_text = ""
file_count = 0
for filename in os.listdir(data_path):
    if filename.endswith(".txt"):
        file_count += 1
        with open(os.path.join(data_path, filename), "r", encoding="utf-8") as file:
            file_content = file.read()
            combined_text += file_content + " "
        print(f"Loaded file: {filename}, Content length: {len(file_content)}")

print(f"\nTotal files processed: {file_count}")
print(f"Total characters in combined text: {len(combined_text)}")

# Print a sample of the combined text
print("\nSample of combined text (first 500 characters):")
print(combined_text[:500])

# Tokenize the text with a sliding window
max_length = 1024
stride = 512
tokenized_data = []

for i in range(0, len(combined_text), stride):
    chunk = combined_text[i:i+max_length]
    tokens = tokenizer(chunk, truncation=True, max_length=max_length, return_tensors="pt")
    tokenized_data.append(tokens.input_ids[0])

print(f"\nNumber of chunks after tokenization: {len(tokenized_data)}")

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        return self.tokenized_data[idx]

def collate_fn(batch):
    # Pad the batch to the maximum length in the batch
    return pad_sequence(batch, batch_first=True, padding_value=tokenizer.pad_token_id)

dataset = TextDataset(tokenized_data)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

print(f"Dataset size: {len(dataset)}")
print(f"First batch shape: {next(iter(dataloader)).shape}")

# Print a sample of tokenized and decoded text
sample_tokens = tokenized_data[0]
print("\nSample of tokenized and decoded text:")
print(tokenizer.decode(sample_tokens))

# Set up optimizer with Gemma-recommended parameters
optimizer = AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.01)

# Enable mixed precision training
scaler = torch.amp.GradScaler('cuda')

# Accumulate gradients over 4 steps (adjust as needed based on your memory constraints)
gradient_accumulation_steps = 4

# Learning rate scheduler
from transformers import get_linear_schedule_with_warmup

num_training_steps = len(dataloader) * 3  # 3 epochs, adjust as needed
num_warmup_steps = num_training_steps // 10  # 10% of total steps for warmup

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

# Verify data loading
print(f"Dataset size: {len(dataset)}")
print(f"First batch shape: {next(iter(dataloader)).shape}")

# Ensure model is in training mode and on correct device
model.train()
print(f"Model device: {next(model.parameters()).device}")

# Fine-tuning loop with detailed logging
for epoch in range(3):  # Consider increasing this
    total_loss = 0
    for i, batch in enumerate(dataloader):
        print(f"Processing batch {i+1}")
        batch = batch.to(device)
        print(f"Batch shape: {batch.shape}, Device: {batch.device}")

        optimizer.zero_grad()

        with torch.amp.autocast(device_type='cuda'):
            outputs = model(input_ids=batch, labels=batch)
            loss = outputs.loss

        print(f"Raw loss: {loss.item()}")

        loss = loss / gradient_accumulation_steps
        scaler.scale(loss).backward()

        if (i + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item()
        if (i + 1) % 10 == 0:  # Print loss more frequently
            print(f"Epoch {epoch+1}, Step {i+1}, Avg Loss: {total_loss / 10:.4f}")
            total_loss = 0

        # Break after a few batches for testing
        if i == 20:
            break

    print(f"Completed epoch {epoch+1}")

print("Training completed")

# Save the fine-tuned model weights
fine_tuned_path = "/content/drive/MyDrive/MTLiens/Refined"
torch.save(model.state_dict(), f"{fine_tuned_path}/pytorch_model.bin")
tokenizer.save_pretrained(fine_tuned_path)

print("Model saved. Now loading for inference...")

# Load the fine-tuned model for inference
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/gemma-2-2b-it")  # Load original model architecture
model.load_state_dict(torch.load(f"{fine_tuned_path}/pytorch_model.bin"))  # Load fine-tuned weights
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_path)

print("Fine-tuned model loaded. Testing...")

# Test the model
test_input = "What is the 7th step?"
inputs = tokenizer(test_input, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_length=500)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Compare with original model
original_model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/gemma-2-2b-it")
original_model.to(device)
outputs = original_model.generate(inputs['input_ids'], max_length=500)
print("Original model output:", tokenizer.decode(outputs[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded file: BigBook no stories no appendices.txt, Content length: 259323
Loaded file: Clean_AsBillSeesIt.txt, Content length: 254165

Total files processed: 2
Total characters in combined text: 513490

Sample of combined text (first 500 characters):
Chapter 1
    
    BILL'S STORY
    
    WAR FEVER ran high in the New England town to which we new, young officers from Plattsburg were assigned, and we were flattered when the first citizens took us to their homes, making us feel heroic.  Here was love, applause, war; moments sublime with intervals hilarious.  I was part of life at last, and in the midst of the excitement I discovered liquor.  I forgot the strong warnings and the prejudices of my people concerning drink.  In time we sailed for

Number of chunks after tokenization: 1003
Dataset size: 1003
First batch shape: torch.Size([2, 263])

Sample of tokenized and decoded text:
<bos>Chapter 1
    
    BILL'S STORY
    
    WAR FEVER ran high in the New England town to which we new, y

OutOfMemoryError: CUDA out of memory. Tried to allocate 82.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 74.81 MiB is free. Process 283492 has 39.48 GiB memory in use. Of the allocated memory 38.44 GiB is allocated by PyTorch, and 553.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

//Claude Low Mem\\ including fp16

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, get_linear_schedule_with_warmup
import torch
from torch.nn.utils.rnn import pad_sequence
import os
from torch.utils.data import DataLoader, Dataset

# Load the tokenizer and model from your local path
model_path = "/content/drive/MyDrive/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_path = "/content/drive/Othercomputers/My Computer (1)/Google Drive Sync/Shared Docs/Cleaned"

# Load and concatenate all text files in the directory
combined_text = ""
file_count = 0
for filename in os.listdir(data_path):
    if filename.endswith(".txt"):
        file_count += 1
        with open(os.path.join(data_path, filename), "r", encoding="utf-8") as file:
            file_content = file.read()
            combined_text += file_content + " "
        print(f"Loaded file: {filename}, Content length: {len(file_content)}")

print(f"\nTotal files processed: {file_count}")
print(f"Total characters in combined text: {len(combined_text)}")

# Print a sample of the combined text
print("\nSample of combined text (first 500 characters):")
print(combined_text[:500])

# Tokenize the text with a sliding window
max_length = 1024
stride = 512
tokenized_data = []

for i in range(0, len(combined_text), stride):
    chunk = combined_text[i:i+max_length]
    tokens = tokenizer(chunk, truncation=True, max_length=max_length, return_tensors="pt")
    tokenized_data.append(tokens.input_ids[0])

print(f"\nNumber of chunks after tokenization: {len(tokenized_data)}")

class TextDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        return self.tokenized_data[idx]

def collate_fn(batch):
    # Pad the batch to the maximum length in the batch
    return pad_sequence(batch, batch_first=True, padding_value=tokenizer.pad_token_id)

dataset = TextDataset(tokenized_data)

# Reduce batch size
batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

print(f"Dataset size: {len(dataset)}")
print(f"First batch shape: {next(iter(dataloader)).shape}")

# Print a sample of tokenized and decoded text
sample_tokens = tokenized_data[0]
print("\nSample of tokenized and decoded text:")
print(tokenizer.decode(sample_tokens))

# Load model with bfloat16 precision and gradient checkpointing
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    use_cache=False,
    attn_implementation='eager'
)
model.gradient_checkpointing_enable()
model.to(device)

print(f"Model device: {next(model.parameters()).device}")

# Optimizer setup
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

# Learning rate scheduler
num_training_steps = len(dataloader) * 3  # 3 epochs
num_warmup_steps = num_training_steps // 10
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# Gradient accumulation steps
gradient_accumulation_steps = 8

# Training loop
model.train()
for epoch in range(3):
    total_loss = 0
    for i, batch in enumerate(dataloader):
        batch = batch.to(device)

        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            outputs = model(input_ids=batch, labels=batch)
            loss = outputs.loss / gradient_accumulation_steps

        loss.backward()

        if (i + 1) % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        total_loss += loss.item() * gradient_accumulation_steps

        if (i + 1) % (10 * gradient_accumulation_steps) == 0:
            print(f"Epoch {epoch+1}, Step {i+1}, Avg Loss: {total_loss / (10 * gradient_accumulation_steps):.4f}")
            total_loss = 0

    print(f"Completed epoch {epoch+1}")

# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/fine_tuned_gemma/o1-iteration/New-Data-Reiteration")
tokenizer.save_pretrained("/content/drive/MyDrive/fine_tuned_gemma/o1-iteration/New-Data-Reiteration")
torch.save(model.state_dict(), "/content/drive/MyDrive/fine_tuned_gemma/o1-iteration/New-Data-Reiteration/pytorch_model.bin")

print("Training completed and model saved.")



Loaded file: p3-ClaudeCleaned.txt, Content length: 4868
Loaded file: p2-ClaudeCleaned.txt, Content length: 61802
Loaded file: p5-Cleaned.txt, Content length: 53317
Loaded file: p83_Cleaned.txt, Content length: 56537
Loaded file: p-1_Claude.txt, Content length: 21015
Loaded file: 12-Traditions.txt, Content length: 1540
Loaded file: p4-ClaudeCleaned.txt, Content length: 21685
Loaded file: DropTheRock_ClaudeCleanedManual.txt, Content length: 123227
Loaded file: Clean_AsBillSeesIt_Corrected.txt, Content length: 251595
Loaded file: BigBook no stories no appendices.txt, Content length: 259360
Loaded file: 12-Steps.txt, Content length: 1231

Total files processed: 11
Total characters in combined text: 856188

Sample of combined text (first 500 characters):
Is A.A. for You?

Only you can decide whether you want to give A.A. a try — whether you think it can help. The questions that follow below may help you make a decision.

We who are in A.A. came because we finally gave up trying to control o

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model device: cuda:0


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch 1, Step 80, Avg Loss: 3.1942
Epoch 1, Step 160, Avg Loss: 3.2093
Epoch 1, Step 240, Avg Loss: 3.2125
Epoch 1, Step 320, Avg Loss: 3.1365
Epoch 1, Step 400, Avg Loss: 3.1198
Epoch 1, Step 480, Avg Loss: 3.1672
Epoch 1, Step 560, Avg Loss: 3.1001
Epoch 1, Step 640, Avg Loss: 3.1073
Epoch 1, Step 720, Avg Loss: 3.0575
Epoch 1, Step 800, Avg Loss: 3.0187
Epoch 1, Step 880, Avg Loss: 3.0054
Epoch 1, Step 960, Avg Loss: 2.9338
Epoch 1, Step 1040, Avg Loss: 2.9216
Epoch 1, Step 1120, Avg Loss: 2.8796
Epoch 1, Step 1200, Avg Loss: 2.9646
Epoch 1, Step 1280, Avg Loss: 2.8337
Epoch 1, Step 1360, Avg Loss: 2.8368
Epoch 1, Step 1440, Avg Loss: 2.8865
Epoch 1, Step 1520, Avg Loss: 2.8542
Epoch 1, Step 1600, Avg Loss: 2.8830
Completed epoch 1
Epoch 2, Step 80, Avg Loss: 2.7552
Epoch 2, Step 160, Avg Loss: 2.7280
Epoch 2, Step 240, Avg Loss: 2.7097
Epoch 2, Step 320, Avg Loss: 2.6916
Epoch 2, Step 400, Avg Loss: 2.6530
Epoch 2, Step 480, Avg Loss: 2.6336
Epoch 2, Step 560, Avg Loss: 2.7216
Epoc

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, get_linear_schedule_with_warmup
import torch
from torch.nn.utils.rnn import pad_sequence
import os
from torch.utils.data import DataLoader, Dataset

# Load the tokenizer and model from your local path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load the fine-tuned model for inference

fine_tuned_model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/fine_tuned_gemma/o1-iteration/New-Data-Reiteration")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/fine_tuned_gemma/o1-iteration/New-Data-Reiteration")
fine_tuned_model.to(device)

# Test the fine-tuned model
test_input = "What is step 1, and how do i exercise it?"
inputs = tokenizer(test_input, return_tensors="pt").to(device)
outputs = fine_tuned_model.generate(inputs['input_ids'], max_length=500)
print("Fine-tuned model output:", tokenizer.decode(outputs[0], skip_special_tokens=True))



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fine-tuned model output: What is step 1, and how do i exercise it?

It is Step One of the Twelve Steps of Alcoholics Anonymous:

**"We admitted we were powerless over alcohol — that our lives had become unmanageable."**

How do I exercise this step?

The first step is the hardest. It requires a complete willingness to change and a willingness to admit that we cannot do it alone. It requires a willingness to look at our lives and see that we have been living in denial. It requires a willingness to look at our lives and see that we have been living in denial. It requires a willingness to look at our lives and see that we have been living in denial. It requires a willingness to look at our lives and see that we have been living in denial. It requires a willingness to look at our lives and see that we have been living in denial. It requires a willingness to look at our lives and see that we have been living in denial. It requires a willingness to look at our lives and see that we have been

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the tokenizer and model from your local path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned model for inference
fine_tuned_model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/fine_tuned_gemma/o1-iteration/New-Data-Reiteration")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/fine_tuned_gemma/o1-iteration/New-Data-Reiteration")
fine_tuned_model.to(device)

# Test the fine-tuned model
test_input = "Respond in clear and concise language. What is Step Three, and how do i practice it?"
inputs = tokenizer(test_input, return_tensors="pt").to(device)

# Generate text with repetition penalty and controlled creativity
outputs = fine_tuned_model.generate(
    inputs['input_ids'],
    max_length=400,  # Adjust based on how long you'd like it to generate
    temperature=0.2,  # Keep it minimally creative
    top_k=20,  # Top-k sampling - next word pool
    top_p=0.8,  # Nucleus sampling
    repetition_penalty=1.2,  # Apply a penalty to discourage repetition
    no_repeat_ngram_size=3,  # Ensure no 3-gram repetition
    early_stopping=True,  # Stops the generation once it feels the output is repetitive or lower quality
    do_sample=True  # Enables sampling to keep creativity
)

# Print the model's output
print("Fine-tuned model output:", tokenizer.decode(outputs[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fine-tuned model output: Respond in clear and concise language. What is Step Three, and how do i practice it?

Twelve Steps are the foundation of recovery for members who identify as alcoholics or addicts. They represent a practical approach to living without addiction by providing tools that help individuals maintain sobriety (or abstinence) from alcohol and other addictive substances. The Twelve Steps offer guidance on developing healthy emotional habits and behaviors necessary for long-term stability in recovery. 

The Third Step says: "We made direct amends to such people where we were wrong; ... We tried to carry this message to alcoholics, and to all who suffer from alcoholism." This means taking responsibility for our actions and apologizing when appropriate. It also involves sharing what we know about alcoholism with others so they can get sober themselves if they want to.


What does practicing Step Three look like in daily life? Here's an example:
You have been late getting t

In [ ]:
# Compare with original model
model_path = "/content/drive/MyDrive/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_path)
original_model = AutoModelForCausalLM.from_pretrained(model_path)
original_model.to(device)
outputs = original_model.generate(inputs['input_ids'], max_length=500)
print("Original model output:", tokenizer.decode(outputs[0], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Original model output: How do i find a Higher Power in my life?

It's a question that has been pondered by people for centuries.  There's no one right answer, as the concept of a Higher Power is deeply personal and can vary widely. 

Here are some ways people find their Higher Power:

**1. Explore Different Spiritual Traditions:**
* **Religion:**  Many religions offer a framework for understanding a Higher Power, such as God, Allah, Buddha, or the Tao.  
* **Spirituality:**  Practices like meditation, yoga, and mindfulness can help you connect with a sense of the divine within yourself.
* **Nature:**  Spending time in nature can be a source of awe and wonder, prompting a sense of connection to something larger than yourself.

**2. Reflect on Your Values and Beliefs:**
* **What do you believe in?** What gives your life meaning? 
* **What are your core values?**  Do they point towards a higher purpose or a sense of interconnectedness?
* **What makes you feel most alive and fulfilled?**  

//Model Reducer\\ --- Reduced by only 20%

In [ ]:
import torch
from transformers import AutoModelForCausalLM
import torch.nn.utils.prune as prune

# Model and Tokenizer loading paths
model_path = '/content/drive/MyDrive/fine_tuned_gemma/o1-iteration'
save_path = '/content/drive/MyDrive/fine_tuned_gemma/pruned_quantized_model.pth'

# Set the device to GPU if available, fallback to CPU if not
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned model on GPU for pruning
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, use_cache=False)

# Move the model to GPU
model.to(device)

print(f"Model loaded on device: {next(model.parameters()).device}")

# Apply light magnitude-based weight pruning to all Linear layers (10% pruning)
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        # Prune the weights with the smallest magnitude, but only 10%
        prune.l1_unstructured(module, name='weight', amount=0.15)

# Remove pruning re-parametrization and make pruning permanent
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.remove(module, 'weight')

print("Light pruning complete (10%).")

# Move the model to CPU for dynamic quantization
model.to("cpu")

# Apply dynamic quantization (quantizes Linear layers to qint8)
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8  # Quantize to qint8 for Linear layers
)

print("Dynamic quantization complete.")

# Save the pruned and quantized model using torch.save
torch.save(quantized_model.state_dict(), save_path)
print(f"Pruned and quantized model saved to {save_path}")




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on device: cuda:0
Light pruning complete (10%).
Dynamic quantization complete.
Pruned and quantized model saved to /content/drive/MyDrive/fine_tuned_gemma/pruned_quantized_model.pth
Pruned and quantized model saved to /content/drive/MyDrive/fine_tuned_gemma/pruned_quantized_model.pth
